<a href="https://colab.research.google.com/github/nagbodo2/it-cert-automation-practice/blob/master/hrAssistant_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28
!pip install pdfplumber
!pip install gradio==4.44.1
import openai
import gradio as gr
import pdfplumber

In [2]:
# Setting OpenAI API key
openai.api_key = 'You api key here'

# Function to extract text from the company's policy document PDFs
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text()
    return all_text


In [3]:

# Loading the policy document once during initialization
policy_pdf_path = "your file path here"
policy_text = extract_text_from_pdf(policy_pdf_path)

In [ ]:
print(policy_text[0:250])

In [5]:
# Function to handle user queries and search for relevant content from the policy document
def hr_assistant(query):
    # If query is found directly in the policy document
    if query.lower() in policy_text.lower():
        start_idx = policy_text.lower().find(query.lower())
        end_idx = start_idx + 1000  # Taking a snippet of text around the query
        response_text = policy_text[start_idx:end_idx]
    else:
        # If query isn't directly found in the policy text, ask GPT to generate a response
        # Updated to use a supported model such as 'text-davinci-002' or 'gpt-3.5-turbo'
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"Provide an answer based on the following policy information:\n\n{policy_text}"},
                {"role": "user", "content": query}
            ],
            max_tokens=250,
            temperature=0.7,
        )
        response_text = response.choices[0].message.content.strip()  # Accessing the response text

    return response_text

In [ ]:
# Testing the prompt
print(hr_assistant("What is the policy about?"))

In [6]:
# Create a Gradio interface for the HR chatbot
iface = gr.Interface(
    fn=hr_assistant,
    inputs="text",
    outputs="text",
    title="HR Assistant Chatbot",
    description="Ask HR-related questions. The assistant will extract answers from the company's policy documents."
)

# Launch the Gradio interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5480ce6c810636b1de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
